In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import cluster, decomposition, metrics

mpl.rcParams["figure.figsize"] = (16, 8)
mpl.rcParams["figure.dpi"] = 160

Here we'll perform a first clustering model, folowing the RFM methodology. So, this notebook will perform some eda on those variables and after run a K-means algorithm to build our clusters.

In [ ]:
orders_df = pd.read_csv("../data/orders.csv", sep=";")
orders_dtypes = pd.read_csv("../data/orders-dtype.csv")
orders_dtypes.set_index(["features"], inplace=True)
dtypes_to_read = orders_dtypes.to_dict()["dtypes"]

# dtypes orders fixed
orders_df = orders_df.astype(dtypes_to_read)

In [ ]:
# variables to use
orders_rfm_df = orders_df[
    ["delivery_fee", "total_amount", "subsidy_amount", "lag_last_order", "account_id"]
]

In [ ]:
# lets look if our data have missings
orders_rfm_df.isnull().sum()

In [ ]:
# curiously the number of less than 0 on subsidy amount is the same
orders_rfm_df[orders_rfm_df["subsidy_amount"] < 0].shape

In [ ]:
# as we know that this subsidy_amount needs to be positive, and we doesn't have
# those accounts, let's drop those values and look if our missings desapeear
orders_rfm_pos_df = orders_rfm_df[orders_rfm_df["subsidy_amount"] >= 0].copy()
orders_rfm_pos_df.isnull().sum()

In [ ]:
# Look, account_id now doesn't have missings
orders_rfm_pos_df[orders_rfm_df["lag_last_order"].isnull()]

In [ ]:
orders_rfm_pos_df.describe(
    percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]
)

In [ ]:
# looking at the lag_last_order missings seems that we have values coehrent,
# and those percentiles also are looking fine.
# but, we're seeing that we have a max value in all of our variables
# let's remove the max value observation for all variables
orders_rfm_pos_wo_max = orders_rfm_pos_df[
    (
        (
            orders_rfm_pos_df["subsidy_amount"]
            < orders_rfm_pos_df["subsidy_amount"].max()
        )
        & (orders_rfm_pos_df["total_amount"] < 140000)
        & (  # this value was obtained from the boxplot
            orders_rfm_pos_df["lag_last_order"]
            < orders_rfm_pos_df["lag_last_order"].max()
        )
        & (orders_rfm_pos_df["delivery_fee"] >= 0)
    )
].copy()

orders_rfm_pos_wo_max.describe(
    percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]
)

In [ ]:
# plotting all boxplots
columns_to_plot = orders_rfm_pos_wo_max.columns
for columns in columns_to_plot:
    if columns != "account_id":
        _ = sns.boxplot(orders_rfm_pos_wo_max[columns])
        plt.show()

In [ ]:
# lets build our new variable total paid (total_amount + subsidy_amount)
orders_rfm_pos_wo_max.loc[:, "total_paid"] = (
    orders_rfm_pos_wo_max["total_amount"] + orders_rfm_pos_wo_max["subsidy_amount"]
)
orders_rfm_pos_wo_max

In [ ]:
orders_cluster_rfm_df = orders_rfm_pos_wo_max.groupby("account_id").agg(
    [np.mean, np.size]
)
orders_cluster_rfm_df

In [ ]:
orders_cluster_to_use = orders_cluster_rfm_df.loc[
    :,
    [
        ("delivery_fee", "mean"),
        ("total_amount", "mean"),
        ("subsidy_amount", "mean"),
        ("lag_last_order", "mean"),
        ("total_paid", "mean"),
        ("total_paid", "size"),
    ],
]
orders_cluster_to_use

In [ ]:
orders_cluster_to_use_df = orders_cluster_to_use.reset_index()
orders_cluster_to_use_df

In [ ]:
orders_de_mean = orders_cluster_to_use_df.loc[:, ("delivery_fee", "mean")]
orders_ta_mean = orders_cluster_to_use_df.loc[:, ("total_amount", "mean")]
orders_sa_mean = orders_cluster_to_use_df.loc[:, ("subsidy_amount", "mean")]
orders_llo_mean = orders_cluster_to_use_df.loc[:, ("lag_last_order", "mean")]
orders_tp_mean = orders_cluster_to_use_df.loc[:, ("total_paid", "mean")]
orders_size = orders_cluster_to_use_df.loc[:, ("total_paid", "size")]

orders_to_cluster = pd.DataFrame(
    {
        "delivery_fee_mean": orders_de_mean,
        "total_amount_mean": orders_ta_mean,
        "subsidy_amount_mean": orders_sa_mean,
        "lag_last_order_mean": orders_llo_mean,
        "total_paid_mean": orders_tp_mean,
        "orders_qt": orders_size,
    }
)

orders_to_cluster

In [ ]:
# let's see how our variables are correlated using a scatter plot
_ = sns.pairplot(orders_to_cluster)

In [ ]:
# now with a log transformed variables
orders_to_cluster_log = orders_to_cluster.apply(lambda x: np.log(x + 1))
_ = sns.pairplot(orders_to_cluster_log)

In [ ]:
# looking at the correlation
corr = orders_to_cluster_log.corr()
corr

In [ ]:
# looking at those missings
msno.matrix(orders_to_cluster_log)

In [ ]:
# as our missings are on the same place, let's drop
orders_to_cluster_log.dropna(axis=0, inplace=True)
msno.matrix(orders_to_cluster_log)

# PCA

In [ ]:
pca = decomposition.PCA(n_components=2)
orders_to_cluster_components = pca.fit_transform(orders_to_cluster_log)
orders_to_cluster_components_df = pd.DataFrame(
    orders_to_cluster_components, columns=["pc1", "pc2"]
)
orders_to_cluster_components_df

# Clustering with k-means

In [ ]:
clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]

for n_cluster in clusters:
    cluster_kmeans = cluster.KMeans(n_clusters=n_cluster).fit(orders_to_cluster_log)
    preds = cluster_kmeans.predict(orders_to_cluster_log)
    centers = cluster_kmeans.cluster_centers_
    score = metrics.silhouette_score(orders_to_cluster_log, preds, metric="euclidean")
    print("For n cluster: {}. The avg silhouette_score is {}".format(n_cluster, score))

In [ ]:
# sems that 6 clusters are interesting
clusters = cluster.KMeans(n_clusters=6).fit(orders_to_cluster_log)
preds = clusters.predict(orders_to_cluster_log)
preds

In [ ]:
# bring pred clusters to our PCA analysis
orders_to_cluster_components_df["clusters"] = preds
orders_to_cluster_components_df

In [ ]:
_ = sns.scatterplot(
    x="pc1",
    y="pc2",
    data=orders_to_cluster_components_df,
    hue="clusters",
    palette=sns.color_palette("tab10", 6),
)
plt.show()

In [ ]:
# copied from kaggle
def pca_results(good_data, pca):
    """
    Create a DataFrame of the PCA results
    Includes dimension feature weights and explained variance
    Visualizes the PCA results
    """

    # Dimension indexing
    dimensions = ["Dimension {}".format(i) for i in range(1, len(pca.components_) + 1)]

    # PCA components
    components = pd.DataFrame(np.round(pca.components_, 4), columns=good_data.keys())
    components.index = dimensions

    # PCA explained variance
    ratios = pca.explained_variance_ratio_.reshape(len(pca.components_), 1)
    variance_ratios = pd.DataFrame(np.round(ratios, 4), columns=["Explained Variance"])
    variance_ratios.index = dimensions

    # Create a bar plot visualization
    fig, ax = plt.subplots(figsize=(14, 8))

    # Plot the feature weights as a function of the components
    components.plot(ax=ax, kind="bar")
    ax.set_ylabel("Feature Weights")
    ax.set_xticklabels(dimensions, rotation=0)

    # Display the explained variance ratios
    for i, ev in enumerate(pca.explained_variance_ratio_):
        ax.text(
            i - 0.40,
            ax.get_ylim()[1] + 0.05,
            "Explained Variance\n          %.4f" % (ev),
        )

    # Return a concatenated DataFrame
    return pd.concat([variance_ratios, components], axis=1)

In [ ]:
pca_results(orders_to_cluster_log, pca)

In [ ]:
orders_df[orders_df["order_origin"] == "STORE"]

In [ ]:
orders_df[orders_df["subsidy_amount"] < 0].value_counts(["order_origin"])

In [ ]:
orders_df.value_counts(["order_origin"])

In [ ]:
orders_df[orders_df["subsidy_amount"] < 0].shape

In [ ]:
# lets imput the median on this less than 0 observations
orders_rfm_df.loc[orders_rfm_df["subsidy_amount"] < 0, "subsidy_amount"] = None
orders_rfm_df

In [ ]:
orders_rfm_df.isnull().sum()

In [ ]:
# as we already saw, the subsidy_amount needs to be positive
# and also, we saw that there's just one single order with a very high value.